# Image Classification Evaluation
### Imports & Config

In [1]:
import sys
import torch
import numpy as np
import yaml
import pandas as pd
from tqdm import tqdm
import random
import json
import os
from collections import OrderedDict

from fromage.imgcls_dataset import RSNAPneumoniaDataset, COVIDDataset
from fromage.data import cxr_image_transform
from fromage.experiment import Experiment

/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rsna_dataset_path = '../data/datasets/RSNA'
covid_dataset_path = "../data/datasets/COVID_QU_EX"

ckpt_path = "../logs/checkpoints/debug_2/last.ckpt"
config_path = "../logs/checkpoints/debug_2/config.yaml"

### Load datasets and model

In [3]:
transform = cxr_image_transform(resize=512, center_crop_size=480, train=False) 

RSNAdataset = RSNAPneumoniaDataset(rsna_dataset_path, transform)
COVIDdataset = COVIDDataset(covid_dataset_path, transform)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open (config_path) as file:
    config = yaml.safe_load(file)

model = Experiment(config, inference=True)

/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/kuacc/users/oince22/.conda/envs/fromage/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


_IncompatibleKeys(missing_keys=['lm.model.decoder.embed_tokens.weight', 'lm.model.decoder.embed_positions.weight', 'lm.model.decoder.final_layer_norm.weight', 'lm.model.decoder.final_layer_norm.bias', 'lm.model.decoder.layers.0.self_attn.k_proj.weight', 'lm.model.decoder.layers.0.self_attn.k_proj.bias', 'lm.model.decoder.layers.0.self_attn.v_proj.weight', 'lm.model.decoder.layers.0.self_attn.v_proj.bias', 'lm.model.decoder.layers.0.self_attn.q_proj.weight', 'lm.model.decoder.layers.0.self_attn.q_proj.bias', 'lm.model.decoder.layers.0.self_attn.out_proj.weight', 'lm.model.decoder.layers.0.self_attn.out_proj.bias', 'lm.model.decoder.layers.0.self_attn_layer_norm.weight', 'lm.model.decoder.layers.0.self_attn_layer_norm.bias', 'lm.model.decoder.layers.0.fc1.weight', 'lm.model.decoder.layers.0.fc1.bias', 'lm.model.decoder.layers.0.fc2.weight', 'lm.model.decoder.layers.0.fc2.bias', 'lm.model.decoder.layers.0.final_layer_norm.weight', 'lm.model.decoder.layers.0.final_layer_norm.bias', 'lm.mod

In [5]:
print(model.model.model.logit_scale)

Parameter containing:
tensor(265.9260, requires_grad=True)


In [6]:
torch.nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

Parameter containing:
tensor(2.6593, requires_grad=True)

In [7]:
model.model.model.load_state_dict(torch.load("/scratch/users/oince22/hpc_run/fromage_scratch/fromage/logs/checkpoints/debug_2/last.ckpt"), strict=False)

_IncompatibleKeys(missing_keys=['lm.model.decoder.embed_tokens.weight', 'lm.model.decoder.embed_positions.weight', 'lm.model.decoder.final_layer_norm.weight', 'lm.model.decoder.final_layer_norm.bias', 'lm.model.decoder.layers.0.self_attn.k_proj.weight', 'lm.model.decoder.layers.0.self_attn.k_proj.bias', 'lm.model.decoder.layers.0.self_attn.v_proj.weight', 'lm.model.decoder.layers.0.self_attn.v_proj.bias', 'lm.model.decoder.layers.0.self_attn.q_proj.weight', 'lm.model.decoder.layers.0.self_attn.q_proj.bias', 'lm.model.decoder.layers.0.self_attn.out_proj.weight', 'lm.model.decoder.layers.0.self_attn.out_proj.bias', 'lm.model.decoder.layers.0.self_attn_layer_norm.weight', 'lm.model.decoder.layers.0.self_attn_layer_norm.bias', 'lm.model.decoder.layers.0.fc1.weight', 'lm.model.decoder.layers.0.fc1.bias', 'lm.model.decoder.layers.0.fc2.weight', 'lm.model.decoder.layers.0.fc2.bias', 'lm.model.decoder.layers.0.final_layer_norm.weight', 'lm.model.decoder.layers.0.final_layer_norm.bias', 'lm.mod

In [8]:
for name, param in model.named_parameters():
    if name == "model.model.logit_scale":
        print(name, param)

model.model.logit_scale Parameter containing:
tensor(265.9260, requires_grad=True)


In [9]:
for name, param in model.named_parameters():
    if name == "model.model.ret_i2t_mapping.weight":
        print(name)
        print(param.size(), param.std())

model.model.ret_i2t_mapping.weight
torch.Size([512, 2048]) tensor(1.2757, grad_fn=<StdBackward0>)


In [5]:
import string 

right_answers = 0
total_answers = 0

rsna_classes = ["yes", "no"]

for idx in tqdm(RSNAdataset):
    img, ans = idx 
    with torch.inference_mode():
        model.eval()
        prompts = [idx[0], "Question: Is pneumonia present in this chest x-ray image? Answer (yes or no): "] 
        model_ans = model.classification_for_eval(prompts, rsna_classes) # top_p=0.9, temperature=0.5
        
        print(model_ans, ans)
        if model_ans.lower() == ans.lower():
            right_answers += 1

        total_answers += 1  

print(right_answers, '/', total_answers )
print((right_answers/total_answers)*100, '% correct')

yes yes
yes no


 14%|█▍        | 493/3538 [01:05<05:20,  9.49it/s]

yes no
yes no


 14%|█▍        | 495/3538 [01:06<05:35,  9.07it/s]

yes no
yes no


...

yes yes
yes no
yes yes


100%|█████████▉| 3535/3538 [06:11<00:00, 10.27it/s]

yes yes
yes no
yes no


100%|██████████| 3538/3538 [06:11<00:00,  9.51it/s]

yes yes
yes yes
yes no
1181 / 3538
33.380440927077444 % correct


In [36]:
param_dict = OrderedDict()
for name, param in model.model.model.state_dict().items():
    if not (name.startswith("lm") or name.startswith("vm")):
        print(name, type(param), param.size())
        param_dict[name] = param


logit_scale <class 'torch.Tensor'> torch.Size([])
input_embeddings.weight <class 'torch.Tensor'> torch.Size([50267, 768])
caption_mapping.weight <class 'torch.Tensor'> torch.Size([768, 2048])
caption_mapping.bias <class 'torch.Tensor'> torch.Size([768])
ret_i2t_mapping.weight <class 'torch.Tensor'> torch.Size([512, 2048])
ret_i2t_mapping.bias <class 'torch.Tensor'> torch.Size([512])
ret_t2i_mapping.weight <class 'torch.Tensor'> torch.Size([512, 768])
ret_t2i_mapping.bias <class 'torch.Tensor'> torch.Size([512])


In [37]:
torch.save(param_dict, "model.ckpt")

In [38]:
!ls -ltrh

total 160M
-rw-r--r-- 1 oince22 domainusers  20K Oct 23 21:49 TestPreprocessing.ipynb
-rw-r--r-- 1 oince22 domainusers  16K Oct 23 21:49 TestLM.ipynb
-rw-r--r-- 1 oince22 domainusers  37K Oct 23 21:49 StartFromage.ipynb
-rw-r--r-- 1 oince22 domainusers  19K Oct 23 21:49 NewBioViL.ipynb
-rw-r--r-- 1 oince22 domainusers  14K Oct 23 21:49 MIMICFromageFormat.ipynb
-rw-r--r-- 1 oince22 domainusers 9.1K Oct 23 21:49 Inference.ipynb
-rw-r--r-- 1 oince22 domainusers 4.9K Oct 23 21:49 DICOMtoJPG.ipynb
-rw-r--r-- 1 oince22 domainusers 282K Oct 23 21:49 biovil_test.ipynb
-rw-r--r-- 1 oince22 domainusers  12K Oct 25 20:03 VQAEval.ipynb
-rw-r--r-- 1 oince22 domainusers 383K Nov  3 17:29 ImgClsEval.ipynb
-rw-r--r-- 1 oince22 domainusers 159M Nov  3 17:31 model.ckpt


In [43]:
s = str(model.model.model.load_state_dict(torch.load("model.ckpt", map_location=device), strict=False))

### RSNA dataset evaluation
The RSNA Pneumonia detection dataset has two classes: pneumonia (1) or no pneumonia (0). 

### COVID dataset evaluation
The COVID dataset has three classes: (1) Normal, (2) COVID-19, and (3) Non-COVID

In [49]:
model.model.model.lm.model.decoder.embed_tokens

Embedding(50267, 768)

In [16]:
img, correct_class = COVIDdataset[0]
prompt = "Question: Choose from the following classes: Normal, COVID-19, Non-Covid. Answer: "
print("Prompt: ", prompt)

with torch.inference_mode():
    model.eval()
    prompts = [img, prompt] 
    print("Predicted class: ", model.generate_for_images_and_texts(prompts, top_p=0.9, temperature=0.5))
    
print("Correct Answer: ", correct_class)

Prompt:  Question: Choose from the following classes: Normal, COVID-19, Non-Covid. Answer: 
Predicted class:  _____. HOC. (HOC) and HOC. (HOC) can be best seen in the following: Lateral view. Chest X-
Correct Answer:  Non-COVID


In [17]:
import string 

right_answers = 0
total_answers = 0


for idx in tqdm(COVIDdataset):
    img, ans = idx 
    with torch.inference_mode():
        model.eval()
        prompts = [idx[0], "Question: Choose if the patient case is Normal, COVID-19, or Non-Covid. Answer: "] 
        for _ in range(4): # try 5 times to get the correct answer
            model_ans = get_model_response(prompts)
            if model_ans.lower() == ans.lower():
                right_answers += 1
                break
            else:
                pass
        total_answers += 1        

print(right_answers, '/', total_answers )
print((right_answers/total_answers)*100, '% correct')

  0%|          | 3/3000 [00:05<1:27:28,  1.75s/it]

KeyboardInterrupt

